In [1]:
# ==================================================
# CELL 1: Import All Required Libraries
# ==================================================

import pandas as pd
import numpy as np
import pickle
import warnings
import re
from datetime import datetime

warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)

print("="*70)
print("🚀 NOTEBOOK 5: DISPUTE RESOLUTION PREDICTION SYSTEM")
print("="*70)
print("\n✅ All libraries imported successfully!")
print("\n📦 This notebook will:")
print("   1. Load your trained model")
print("   2. Create prediction functions")
print("   3. Test with sample complaints")
print("   4. Generate prediction reports")
print("="*70)

🚀 NOTEBOOK 5: DISPUTE RESOLUTION PREDICTION SYSTEM

✅ All libraries imported successfully!

📦 This notebook will:
   1. Load your trained model
   2. Create prediction functions
   3. Test with sample complaints
   4. Generate prediction reports


In [2]:
# ==================================================
# CELL 2: Load Saved Model and Components
# ==================================================

print("📂 LOADING SAVED MODEL COMPONENTS")
print("="*70)

models_dir = '../models/'

# Load all components
print("\n🔄 Loading components...")

try:
    # 1. Load model
    print("\n1️⃣ Loading trained model...")
    with open(f'{models_dir}dispute_resolution_model_latest.pkl', 'rb') as f:
        model = pickle.load(f)
    print(f"   ✅ Model loaded: {type(model).__name__}")
    
    # 2. Load TF-IDF vectorizer
    print("\n2️⃣ Loading TF-IDF vectorizer...")
    with open(f'{models_dir}tfidf_vectorizer_latest.pkl', 'rb') as f:
        tfidf_vectorizer = pickle.load(f)
    print(f"   ✅ Vectorizer loaded: {tfidf_vectorizer.max_features} features")
    
    # 3. Load target encoder
    print("\n3️⃣ Loading target encoder...")
    with open(f'{models_dir}target_encoder_latest.pkl', 'rb') as f:
        target_encoder = pickle.load(f)
    print(f"   ✅ Target encoder loaded: {len(target_encoder.classes_)} classes")
    print(f"   Classes: {list(target_encoder.classes_)}")
    
    # 4. Load feature info
    print("\n4️⃣ Loading feature info...")
    with open(f'{models_dir}feature_info_latest.pkl', 'rb') as f:
        feature_info = pickle.load(f)
    print(f"   ✅ Feature info loaded")
    print(f"   Numerical features: {feature_info['numerical_features']}")
    
    # 5. Load metadata
    print("\n5️⃣ Loading model metadata...")
    with open(f'{models_dir}model_metadata_latest.pkl', 'rb') as f:
        metadata = pickle.load(f)
    print(f"   ✅ Metadata loaded")
    print(f"   Model: {metadata['model_name']}")
    print(f"   Accuracy: {metadata['accuracy']:.4f} ({metadata['accuracy']*100:.2f}%)")
    print(f"   F1-Score: {metadata['f1_score']:.4f}")
    
    print("\n" + "="*70)
    print("✅ ALL COMPONENTS LOADED SUCCESSFULLY!")
    print("="*70)
    h
except FileNotFoundError as e:
    print(f"\n❌ ERROR: Could not find model files!")
    print(f"   {e}")
    print("\n   Make sure you completed Notebook 4 and saved the model.")
    
except Exception as e:
    print(f"\n❌ ERROR loading components: {e}")

📂 LOADING SAVED MODEL COMPONENTS

🔄 Loading components...

1️⃣ Loading trained model...
   ✅ Model loaded: LinearSVC

2️⃣ Loading TF-IDF vectorizer...
   ✅ Vectorizer loaded: 3000 features

3️⃣ Loading target encoder...
   ✅ Target encoder loaded: 3 classes
   Classes: ['favor_seller', 'favour_customer', 'split_payment']

4️⃣ Loading feature info...
   ✅ Feature info loaded
   Numerical features: ['processed_word_count', 'negative_word_count', 'positive_word_count', 'urgency_indicator', 'financial_terms_count', 'question_count', 'exclamation_count']

5️⃣ Loading model metadata...
   ✅ Metadata loaded
   Model: Linear SVC
   Accuracy: 0.8192 (81.92%)
   F1-Score: 0.7663

✅ ALL COMPONENTS LOADED SUCCESSFULLY!


In [3]:
# ==================================================
# CELL 3: Define Text Preprocessing Functions
# ==================================================

print("🧹 DEFINING TEXT PREPROCESSING FUNCTIONS")
print("="*70)

def clean_text(text):
    """
    Clean and preprocess complaint text
    SAME function used in training (Notebook 3)
    """
    if pd.isna(text) or text == '':
        return ''
    
    # Convert to string
    text = str(text)
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove XXXX patterns (redaction)
    text = re.sub(r'x{2,}', '', text)
    
    # Remove URLs
    text = re.sub(r'http\S+|www\.\S+', '', text)
    
    # Remove email addresses
    text = re.sub(r'\S+@\S+', '', text)
    
    # Remove phone numbers
    text = re.sub(r'\d{3}[-.\s]?\d{3}[-.\s]?\d{4}', '', text)
    text = re.sub(r'\(\d{3}\)\s*\d{3}[-.\s]?\d{4}', '', text)
    
    # Remove account numbers
    text = re.sub(r'(account|acct|acc)[\s#]*\d+', '', text)
    
    # Remove dates
    text = re.sub(r'\d{1,2}[-/]\d{1,2}[-/]\d{2,4}', '', text)
    
    # Replace currency amounts with placeholder
    text = re.sub(r'\$\s?\d+[\d,]*\.?\d*', 'AMOUNT', text)
    
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text)
    
    # Remove special characters but keep basic punctuation
    text = re.sub(r'[^a-zA-Z0-9\s.,!?]', '', text)
    
    # Trim
    text = text.strip()
    
    return text

def remove_stopwords(text):
    """
    Remove common English stopwords
    SAME function used in training (Notebook 3)
    """
    stopwords = {
        'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 
        'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself',
        'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them',
        'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this',
        'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been',
        'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing',
        'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until',
        'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between',
        'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to',
        'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again',
        'further', 'then', 'once'
    }
    
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stopwords]
    return ' '.join(filtered_words)

def extract_text_features(text):
    """
    Extract numerical features from text
    SAME features used in training (Notebook 3)
    """
    # Negative words
    negative_words = ['terrible', 'horrible', 'worst', 'awful', 'bad', 'poor', 
                     'unacceptable', 'frustrated', 'angry', 'disappointed', 'never']
    negative_count = sum(1 for word in text.split() if word in negative_words)
    
    # Positive words
    positive_words = ['good', 'great', 'excellent', 'satisfied', 'happy', 
                     'resolved', 'helpful', 'thank']
    positive_count = sum(1 for word in text.split() if word in positive_words)
    
    # Urgency words
    urgency_words = ['urgent', 'immediately', 'asap', 'emergency', 'critical', 
                    'important', 'serious', 'severe']
    urgency_indicator = 1 if any(word in text.split() for word in urgency_words) else 0
    
    # Financial terms
    financial_terms = ['payment', 'amount', 'fee', 'charge', 'refund', 'money', 
                      'balance', 'account', 'credit', 'debit', 'transaction']
    financial_count = sum(1 for word in text.split() if word in financial_terms)
    
    # Question marks
    question_count = text.count('?')
    
    # Exclamation marks
    exclamation_count = text.count('!')
    
    # Word count
    word_count = len(text.split())
    
    return {
        'processed_word_count': word_count,
        'negative_word_count': negative_count,
        'positive_word_count': positive_count,
        'urgency_indicator': urgency_indicator,
        'financial_terms_count': financial_count,
        'question_count': question_count,
        'exclamation_count': exclamation_count
    }

print("✅ Text preprocessing functions defined:")
print("   • clean_text() - Removes noise and special characters")
print("   • remove_stopwords() - Removes common words")
print("   • extract_text_features() - Extracts numerical features")
print("\n⚡ These are the SAME functions used during training")
print("="*70)

🧹 DEFINING TEXT PREPROCESSING FUNCTIONS
✅ Text preprocessing functions defined:
   • clean_text() - Removes noise and special characters
   • remove_stopwords() - Removes common words
   • extract_text_features() - Extracts numerical features

⚡ These are the SAME functions used during training


In [4]:
# ==================================================
# CELL 4: Create Main Prediction Function
# ==================================================

print("🎯 CREATING MAIN PREDICTION FUNCTION")
print("="*70)

def predict_dispute_resolution(complaint_text):
    """
    Main function to predict dispute resolution outcome
    
    Args:
        complaint_text (str): Raw complaint text from customer
        
    Returns:
        dict: Prediction results with confidence scores
    """
    
    # Step 1: Clean the text
    cleaned_text = clean_text(complaint_text)
    
    # Step 2: Remove stopwords
    processed_text = remove_stopwords(cleaned_text)
    
    # Check if text is empty after processing
    if not processed_text or len(processed_text) < 5:
        return {
            'error': 'Text too short or empty after preprocessing',
            'original_text': complaint_text[:100]
        }
    
    # Step 3: Extract text features
    text_features = extract_text_features(processed_text)
    
    # Step 4: Convert text to TF-IDF features
    text_tfidf = tfidf_vectorizer.transform([processed_text])
    
    # Step 5: Create numerical features array
    numerical_features_values = np.array([[
        text_features['processed_word_count'],
        text_features['negative_word_count'],
        text_features['positive_word_count'],
        text_features['urgency_indicator'],
        text_features['financial_terms_count'],
        text_features['question_count'],
        text_features['exclamation_count']
    ]])
    
    # Step 6: Combine features (keep sparse)
    from scipy.sparse import csr_matrix, hstack as sparse_hstack
    numerical_sparse = csr_matrix(numerical_features_values)
    X_combined = sparse_hstack([text_tfidf, numerical_sparse])
    
    # Step 7: Make prediction
    prediction_encoded = model.predict(X_combined)[0]
    prediction_label = target_encoder.inverse_transform([prediction_encoded])[0]
    
    # Step 8: Get confidence scores (if model supports decision_function)
    try:
        decision_scores = model.decision_function(X_combined)[0]
        
        # Convert to confidence percentages
        # For multiclass SVC, decision_function returns scores for each class
        if len(decision_scores) == len(target_encoder.classes_):
            # Softmax-like transformation
            exp_scores = np.exp(decision_scores - np.max(decision_scores))
            confidence_scores = exp_scores / exp_scores.sum()
            
            confidence_dict = {}
            for label, score in zip(target_encoder.classes_, confidence_scores):
                confidence_dict[label] = float(score)
        else:
            confidence_dict = {prediction_label: 1.0}
    except:
        # If decision_function not available, just show predicted class
        confidence_dict = {prediction_label: 1.0}
    
    # Step 9: Return results
    return {
        'prediction': prediction_label,
        'confidence_scores': confidence_dict,
        'text_features': text_features,
        'cleaned_text': cleaned_text[:200],
        'processed_text': processed_text[:200]
    }

print("✅ Main prediction function created: predict_dispute_resolution()")
print("\n📝 Usage:")
print("   result = predict_dispute_resolution('Your complaint text here')")
print("="*70)

🎯 CREATING MAIN PREDICTION FUNCTION
✅ Main prediction function created: predict_dispute_resolution()

📝 Usage:
   result = predict_dispute_resolution('Your complaint text here')


In [5]:
# ==================================================
# CELL 5: Create Pretty Print Function for Results
# ==================================================

def print_prediction_result(result):
    """
    Pretty print prediction results
    """
    if 'error' in result:
        print("❌ ERROR:", result['error'])
        return
    
    print("="*70)
    print(" 🎯 DISPUTE RESOLUTION PREDICTION")
    print("="*70)
    
    # Show prediction
    prediction = result['prediction']
    print(f"\n🏆 PREDICTED OUTCOME: {prediction.upper()}")
    
    # Show confidence scores
    print(f"\n📊 CONFIDENCE SCORES:")
    for label, score in sorted(result['confidence_scores'].items(), 
                               key=lambda x: x[1], reverse=True):
        bar_length = int(score * 40)
        bar = '█' * bar_length + '░' * (40 - bar_length)
        print(f"   {label:20s} {bar} {score*100:.2f}%")
    
    # Show text features
    print(f"\n📈 TEXT FEATURES:")
    features = result['text_features']
    print(f"   • Word count: {features['processed_word_count']}")
    print(f"   • Negative words: {features['negative_word_count']}")
    print(f"   • Positive words: {features['positive_word_count']}")
    print(f"   • Financial terms: {features['financial_terms_count']}")
    print(f"   • Urgency: {'Yes' if features['urgency_indicator'] else 'No'}")
    print(f"   • Questions: {features['question_count']}")
    print(f"   • Exclamations: {features['exclamation_count']}")
    
    # Show interpretation
    print(f"\n💡 INTERPRETATION:")
    if prediction == 'favour_customer':
        print("   → Customer should WIN this dispute")
        print("   → Recommendation: Provide refund or relief to customer")
    elif prediction == 'favor_seller':
        print("   → Seller/Company should WIN this dispute")
        print("   → Recommendation: Uphold company position")
    else:  # split_payment
        print("   → COMPROMISE recommended")
        print("   → Recommendation: Partial refund or non-monetary relief")
    
    print("\n" + "="*70)

print("✅ Pretty print function created: print_prediction_result()")
print("="*70)

✅ Pretty print function created: print_prediction_result()


In [6]:
# ==================================================
# CELL 6: Test Prediction - Sample 1
# ==================================================

print("🧪 TEST 1: Customer Should Win")
print("="*70)

sample_complaint_1 = """
I opened a credit card account with this company 3 years ago. I always paid 
my bills on time and never missed a payment. Last month, they charged me a 
$50 late fee even though I paid 2 days before the due date. I have proof of 
payment from my bank statement. When I called customer service, they were 
rude and refused to remove the fee. This is completely unacceptable and unfair. 
I want my $50 refunded immediately. I have been a loyal customer and this 
treatment is terrible. I will never use this company again.
"""

print("\n📝 COMPLAINT TEXT:")
print(sample_complaint_1.strip())

print("\n🔄 Making prediction...")
result = predict_dispute_resolution(sample_complaint_1)

print_prediction_result(result)

🧪 TEST 1: Customer Should Win

📝 COMPLAINT TEXT:
I opened a credit card account with this company 3 years ago. I always paid 
my bills on time and never missed a payment. Last month, they charged me a 
$50 late fee even though I paid 2 days before the due date. I have proof of 
payment from my bank statement. When I called customer service, they were 
rude and refused to remove the fee. This is completely unacceptable and unfair. 
I want my $50 refunded immediately. I have been a loyal customer and this 
treatment is terrible. I will never use this company again.

🔄 Making prediction...
 🎯 DISPUTE RESOLUTION PREDICTION

🏆 PREDICTED OUTCOME: FAVOUR_CUSTOMER

📊 CONFIDENCE SCORES:
   favour_customer      ███████████████░░░░░░░░░░░░░░░░░░░░░░░░░ 39.58%
   favor_seller         ███████████████░░░░░░░░░░░░░░░░░░░░░░░░░ 39.41%
   split_payment        ████████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ 21.01%

📈 TEXT FEATURES:
   • Word count: 56
   • Negative words: 3
   • Positive words: 0
   • Financia

In [7]:
# ==================================================
# CELL 7: Test Prediction - Sample 2
# ==================================================

print("🧪 TEST 2: Seller Should Win")
print("="*70)

sample_complaint_2 = """
I applied for a loan and was denied. The company said I don't meet their 
credit requirements. I think this is wrong but when I look at my credit 
report, I see that I have several late payments and my credit score is 
below 600. The company clearly explained their requirements on their 
website before I applied. I understand they have lending standards.
"""

print("\n📝 COMPLAINT TEXT:")
print(sample_complaint_2.strip())

print("\n🔄 Making prediction...")
result = predict_dispute_resolution(sample_complaint_2)

print_prediction_result(result)

🧪 TEST 2: Seller Should Win

📝 COMPLAINT TEXT:
I applied for a loan and was denied. The company said I don't meet their 
credit requirements. I think this is wrong but when I look at my credit 
report, I see that I have several late payments and my credit score is 
below 600. The company clearly explained their requirements on their 
website before I applied. I understand they have lending standards.

🔄 Making prediction...
 🎯 DISPUTE RESOLUTION PREDICTION

🏆 PREDICTED OUTCOME: FAVOR_SELLER

📊 CONFIDENCE SCORES:
   favor_seller         ██████████████████████████░░░░░░░░░░░░░░ 66.09%
   favour_customer      ███████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ 19.43%
   split_payment        █████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ 14.48%

📈 TEXT FEATURES:
   • Word count: 31
   • Negative words: 0
   • Positive words: 0
   • Financial terms: 3
   • Urgency: No
   • Questions: 0
   • Exclamations: 0

💡 INTERPRETATION:
   → Seller/Company should WIN this dispute
   → Recommendation: Uphold company pos

In [8]:
# ==================================================
# CELL 8: Test Prediction - Sample 3
# ==================================================

print("🧪 TEST 3: Compromise/Split Payment")
print("="*70)

sample_complaint_3 = """
I purchased a product that arrived damaged. The company offered to replace 
it but I would have to pay for return shipping which costs $30. I think 
they should cover the return shipping since the product was damaged when 
it arrived. However, I understand they have a return policy. Maybe we can 
split the cost?
"""

print("\n📝 COMPLAINT TEXT:")
print(sample_complaint_3.strip())

print("\n🔄 Making prediction...")
result = predict_dispute_resolution(sample_complaint_3)

print_prediction_result(result)

🧪 TEST 3: Compromise/Split Payment

📝 COMPLAINT TEXT:
I purchased a product that arrived damaged. The company offered to replace 
it but I would have to pay for return shipping which costs $30. I think 
they should cover the return shipping since the product was damaged when 
it arrived. However, I understand they have a return policy. Maybe we can 
split the cost?

🔄 Making prediction...
 🎯 DISPUTE RESOLUTION PREDICTION

🏆 PREDICTED OUTCOME: FAVOR_SELLER

📊 CONFIDENCE SCORES:
   favor_seller         ███████████████████████░░░░░░░░░░░░░░░░░ 59.71%
   favour_customer      ███████████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ 27.70%
   split_payment        █████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ 12.60%

📈 TEXT FEATURES:
   • Word count: 31
   • Negative words: 0
   • Positive words: 0
   • Financial terms: 0
   • Urgency: No
   • Questions: 1
   • Exclamations: 0

💡 INTERPRETATION:
   → Seller/Company should WIN this dispute
   → Recommendation: Uphold company position



In [9]:
# ==================================================
# CELL 9: Create Batch Prediction Function
# ==================================================

print("📦 CREATING BATCH PREDICTION FUNCTION")
print("="*70)

def predict_batch_complaints(complaints_list):
    """
    Predict outcomes for multiple complaints at once
    
    Args:
        complaints_list: List of complaint texts or DataFrame with 'complaint_text' column
        
    Returns:
        DataFrame with predictions
    """
    
    # Convert to DataFrame if it's a list
    if isinstance(complaints_list, list):
        df = pd.DataFrame({'complaint_text': complaints_list})
    else:
        df = complaints_list.copy()
    
    print(f"🔄 Processing {len(df)} complaints...")
    
    results = []
    
    for idx, row in df.iterrows():
        complaint_text = row['complaint_text'] if 'complaint_text' in row else row[0]
        
        # Make prediction
        result = predict_dispute_resolution(complaint_text)
        
        if 'error' in result:
            results.append({
                'complaint_id': idx,
                'prediction': 'ERROR',
                'confidence': 0,
                'error': result['error']
            })
        else:
            max_confidence = max(result['confidence_scores'].values())
            results.append({
                'complaint_id': idx,
                'complaint_text': complaint_text[:100] + '...',
                'prediction': result['prediction'],
                'confidence': max_confidence,
                'word_count': result['text_features']['processed_word_count'],
                'negative_words': result['text_features']['negative_word_count'],
                'positive_words': result['text_features']['positive_word_count']
            })
        
        # Progress indicator
        if (idx + 1) % 10 == 0:
            print(f"   Processed {idx + 1}/{len(df)} complaints...")
    
    results_df = pd.DataFrame(results)
    print(f"\n✅ Batch prediction complete!")
    
    return results_df

print("✅ Batch prediction function created: predict_batch_complaints()")
print("\n📝 Usage:")
print("   complaints = ['complaint 1', 'complaint 2', 'complaint 3']")
print("   results_df = predict_batch_complaints(complaints)")
print("="*70)

📦 CREATING BATCH PREDICTION FUNCTION
✅ Batch prediction function created: predict_batch_complaints()

📝 Usage:
   complaints = ['complaint 1', 'complaint 2', 'complaint 3']
   results_df = predict_batch_complaints(complaints)


In [10]:
# ==================================================
# CELL 10: Test Batch Prediction
# ==================================================

print("🧪 TESTING BATCH PREDICTION")
print("="*70)

# Create sample batch of complaints
test_complaints = [
    "Company charged me fees I did not authorize. This is fraud! I want full refund.",
    "Applied for credit card but was denied due to low credit score. I understand.",
    "Product was damaged. Company will replace but I need to pay shipping. Not fair.",
    "Been waiting 3 months for refund. Called customer service 10 times. Very frustrated!",
    "Service was okay but could be better. No major issues though."
]

print(f"\n📝 Processing {len(test_complaints)} sample complaints...\n")

# Run batch prediction
results_df = predict_batch_complaints(test_complaints)

print("\n📊 BATCH PREDICTION RESULTS:")
print("="*70)
print(results_df.to_string(index=False))

print("\n📈 SUMMARY STATISTICS:")
print("-"*70)
print(results_df['prediction'].value_counts())

print(f"\nAverage confidence: {results_df['confidence'].mean():.2%}")

print("\n" + "="*70)

🧪 TESTING BATCH PREDICTION

📝 Processing 5 sample complaints...

🔄 Processing 5 complaints...

✅ Batch prediction complete!

📊 BATCH PREDICTION RESULTS:
 complaint_id                                                                          complaint_text   prediction  confidence  word_count  negative_words  positive_words
            0      Company charged me fees I did not authorize. This is fraud! I want full refund.... favor_seller    0.508967           9               0               0
            1        Applied for credit card but was denied due to low credit score. I understand.... favor_seller    0.662264           9               0               0
            2      Product was damaged. Company will replace but I need to pay shipping. Not fair.... favor_seller    0.623196          10               0               0
            3 Been waiting 3 months for refund. Called customer service 10 times. Very frustrated!... favor_seller    0.547913          11               0         

In [11]:
# ==================================================
# CELL 11: Create Function to Save Predictions
# ==================================================

print("💾 CREATING PREDICTION EXPORT FUNCTION")
print("="*70)

def save_predictions(results_df, filename=None):
    """
    Save prediction results to CSV file
    """
    
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f'predictions_{timestamp}.csv'
    
    output_path = f'../data/processed/{filename}'
    
    results_df.to_csv(output_path, index=False)
    
    print(f"✅ Predictions saved to: {output_path}")
    print(f"   • Records: {len(results_df)}")
    print(f"   • File size: {pd.read_csv(output_path).memory_usage(deep=True).sum() / 1024:.2f} KB")
    
    return output_path

print("✅ Export function created: save_predictions()")
print("\n📝 Usage:")
print("   save_predictions(results_df)")
print("   save_predictions(results_df, 'my_predictions.csv')")
print("="*70)

💾 CREATING PREDICTION EXPORT FUNCTION
✅ Export function created: save_predictions()

📝 Usage:
   save_predictions(results_df)
   save_predictions(results_df, 'my_predictions.csv')


In [12]:
# ==================================================
# CELL 12: Interactive Prediction Interface
# ==================================================

print("🎮 INTERACTIVE PREDICTION INTERFACE")
print("="*70)

def interactive_predict():
    """
    Interactive function to get predictions for custom complaints
    """
    
    print("\n" + "="*70)
    print(" 🎯 ESCROW DISPUTE RESOLUTION PREDICTOR")
    print("="*70)
    print("\nEnter a complaint text to get a prediction.")
    print("Type 'quit' to exit.\n")
    
    while True:
        print("-"*70)
        complaint = input("Enter complaint text: ")
        
        if complaint.lower() == 'quit':
            print("\n👋 Goodbye!")
            break
        
        if len(complaint) < 10:
            print("⚠️ Complaint too short! Please enter at least 10 characters.\n")
            continue
        
        print("\n🔄 Processing...")
        result = predict_dispute_resolution(complaint)
        print_prediction_result(result)
        
        print("\n")

print("✅ Interactive interface created: interactive_predict()")
print("\n📝 To use the interactive predictor, run:")
print("   interactive_predict()")
print("\n⚠️ Note: This works in Jupyter notebooks with input support")
print("="*70)

🎮 INTERACTIVE PREDICTION INTERFACE
✅ Interactive interface created: interactive_predict()

📝 To use the interactive predictor, run:
   interactive_predict()

⚠️ Note: This works in Jupyter notebooks with input support


In [13]:
# ==================================================
# CELL 13: Complete Summary and Usage Guide
# ==================================================

print("="*70)
print(" ✅ PREDICTION SYSTEM COMPLETE!")
print("="*70)

print("\n🎯 YOUR DISPUTE RESOLUTION MODEL:\n")

print(f"📊 Model Performance:")
print(f"   • Model: {metadata['model_name']}")
print(f"   • Accuracy: {metadata['accuracy']:.2%}")
print(f"   • F1-Score: {metadata['f1_score']:.4f}")
print(f"   • Training samples: {metadata['training_samples']:,}")

print(f"\n🎨 Prediction Classes:")
for label in target_encoder.classes_:
    print(f"   • {label}")

print(f"\n📝 HOW TO USE THIS SYSTEM:\n")

print("1️⃣ SINGLE PREDICTION:")
print("   complaint = 'Your complaint text here'")
print("   result = predict_dispute_resolution(complaint)")
print("   print_prediction_result(result)")

print("\n2️⃣ BATCH PREDICTION:")
print("   complaints = ['complaint 1', 'complaint 2', 'complaint 3']")
print("   results_df = predict_batch_complaints(complaints)")
print("   print(results_df)")

print("\n3️⃣ SAVE PREDICTIONS:")
print("   save_predictions(results_df, 'my_predictions.csv')")

print("\n4️⃣ INTERACTIVE MODE:")
print("   interactive_predict()")

print("\n" + "="*70)
print("🎉 CONGRATULATIONS! YOUR MODEL IS READY FOR PRODUCTION!")
print("="*70)

print("\n💡 NEXT STEPS:\n")
print("   • Test with real complaints from your dataset")
print("   • Deploy as a web API (Flask/FastAPI)")
print("   • Create a user interface")
print("   • Monitor model performance over time")
print("   • Retrain with new data periodically")

print("\n" + "="*70)

 ✅ PREDICTION SYSTEM COMPLETE!

🎯 YOUR DISPUTE RESOLUTION MODEL:

📊 Model Performance:
   • Model: Linear SVC
   • Accuracy: 81.92%
   • F1-Score: 0.7663
   • Training samples: 125,250

🎨 Prediction Classes:
   • favor_seller
   • favour_customer
   • split_payment

📝 HOW TO USE THIS SYSTEM:

1️⃣ SINGLE PREDICTION:
   complaint = 'Your complaint text here'
   result = predict_dispute_resolution(complaint)
   print_prediction_result(result)

2️⃣ BATCH PREDICTION:
   complaints = ['complaint 1', 'complaint 2', 'complaint 3']
   results_df = predict_batch_complaints(complaints)
   print(results_df)

3️⃣ SAVE PREDICTIONS:
   save_predictions(results_df, 'my_predictions.csv')

4️⃣ INTERACTIVE MODE:
   interactive_predict()

🎉 CONGRATULATIONS! YOUR MODEL IS READY FOR PRODUCTION!

💡 NEXT STEPS:

   • Test with real complaints from your dataset
   • Deploy as a web API (Flask/FastAPI)
   • Create a user interface
   • Monitor model performance over time
   • Retrain with new data periodically


In [14]:
# ==================================================
# CELL 14: Test with Real Data from Your Dataset
# ==================================================

print("🧪 TESTING WITH REAL DATA FROM YOUR DATASET")
print("="*70)

# Load a small sample from your cleaned data
try:
    print("\n🔄 Loading sample from your cleaned dataset...")
    df_test = pd.read_csv('../data/processed/complaints_cleaned.csv', nrows=10)
    
    print(f"✅ Loaded {len(df_test)} sample complaints")
    
    # Get actual labels and predictions
    print("\n🔄 Making predictions on real data...")
    
    comparison_results = []
    
    for idx, row in df_test.iterrows():
        complaint_text = row['complaint_text_processed']
        actual_label = row['dispute_resolution']
        
        # Make prediction
        result = predict_dispute_resolution(complaint_text)
        
        if 'error' not in result:
            predicted_label = result['prediction']
            confidence = max(result['confidence_scores'].values())
            
            comparison_results.append({
                'actual': actual_label,
                'predicted': predicted_label,
                'match': '✅' if actual_label == predicted_label else '❌',
                'confidence': f'{confidence:.2%}'
            })
    
    comparison_df = pd.DataFrame(comparison_results)
    
    print("\n📊 ACTUAL vs PREDICTED:")
    print("="*70)
    print(comparison_df.to_string(index=False))
    
    # Calculate accuracy
    accuracy = (comparison_df['match'] == '✅').sum() / len(comparison_df)
    print(f"\n📈 Sample Accuracy: {accuracy:.2%}")
    
    print("\n" + "="*70)
    
except FileNotFoundError:
    print("\n⚠️ Could not find cleaned dataset file")
    print("   This is optional - you can skip this cell")
    
except Exception as e:
    print(f"\n⚠️ Error: {e}")

🧪 TESTING WITH REAL DATA FROM YOUR DATASET

🔄 Loading sample from your cleaned dataset...
✅ Loaded 10 sample complaints

🔄 Making predictions on real data...

📊 ACTUAL vs PREDICTED:
         actual    predicted match confidence
favour_customer favor_seller     ❌     56.01%
   favor_seller favor_seller     ✅     74.33%
   favor_seller favor_seller     ✅     77.70%
   favor_seller favor_seller     ✅     55.40%
   favor_seller favor_seller     ✅     62.79%
favour_customer favor_seller     ❌     70.19%
   favor_seller favor_seller     ✅     75.11%
   favor_seller favor_seller     ✅     68.45%
   favor_seller favor_seller     ✅     76.83%
   favor_seller favor_seller     ✅     71.08%

📈 Sample Accuracy: 80.00%

